In [ ]:
%run ANNOTATIONS.ipynb

In [ ]:
%run PATTERN_OBSERVER.ipynb

In [ ]:
from ipywidgets import Text,Output,GridspecLayout # for SINGLE_TAB
from ipywidgets import Tab,Layout # for TAB_COLLECTION

In [ ]:
import pickle
a_file = open("data.pkl", "rb")
output = pickle.load(a_file)

In [ ]:
print(len(output))
print(list(output.keys()))

In [ ]:
import pandas as pd

In [ ]:
output['AZUL4.SA']

In [ ]:
def set_index_extraday(df):
    return df.set_index('timestamp')

In [ ]:
xxx=pd.concat(map(set_index_extraday,output.values()),keys=list(output.keys()))
xxx

In [ ]:
pd.concat([output['AZUL4.SA'].set_index(['timestamp','split_coefficient'])],keys='ab')

In [ ]:
multi_index=pd.MultiIndex.from_product([['AZUL4.SA'],output['AZUL4.SA'].timestamp], names=['backtest_name','timestamp'])
out2=output['AZUL4.SA'].set_index(multi_index)
out2

In [ ]:
out2.loc['AZUL4.SA']

In [ ]:
'''
doesnt contain data
only explains how to generate a plot off a pd.DataFrame
'''
class SINGLE_PLOT:
    def __init__(self,
        name        : str              = 'SINGLE_PLOT',
        description : str              = 'SINGLE_PLOT_DESCRIPTION',
        query      = None,
        ):
        self.name=name
        self._query=query
        self._times_drawn=0

        # build widget
        #self.widget=Output(layout=Layout(height='300px',width='10.5%'))
        self.widget=Output()
        self._draw()
    
    # draw
    def _draw(self,data='init') -> None :
        with self.widget:
            # plot stuff here using self._query
            out=Text(self.name+'-'+str(data)+'-'+str(self._times_drawn))
            display(out)
            
    # clear and draw with given data
    def update_plot(self,data) -> None :
        self._times_drawn+=1 # increment counter
        self.widget.clear_output() # clear image
        self._draw(data)
        #print(self.name,'update_plot',self._times_drawn)
        
# example
SINGLE_PLOT('plotname').widget

In [ ]:
'''

a=SINGLE_PLOT()

display(a.widget)

a.update_plot('abc')

a.update_plot('fdsa')

'''
None

In [ ]:
'''
doesnt contain data
only explains how to plots within it generate plots off a pd.DataFrame
'''
class SINGLE_TAB:
    def __init__(self,
        name        : str              = 'SINGLE_TAB',
        description : str              = 'SINGLE_TAB_DESCRIPTION',
        cols        : int              = 3,
        rows        : int              = 3,
        plots       : LOCATIONED_PLOTS = None,
        ):
        self.name         = name
        self.description  = description
        self._cols        = cols
        self._rows        = rows
        self._plots       = plots if not plots==None else self._populate_defaults_plots() # populate default
        self._times_drawn = 0
        self.widget       = self._make()
        
    ####################################
    # populate defaults
    ####################################
    def _populate_defaults_plots(self):
        out={}
        for r in range(self._rows):
            for c in range(self._cols):
                out[(r,c)]=SINGLE_PLOT(name=self.name+'['+str(r)+str(c)+']')
        return out

    def _make(self) -> GridspecLayout :
        # instantiate
        gridspec=GridspecLayout(n_rows=self._rows,n_columns=self._cols)
        
        # populate gridspec
        for r in range(self._rows):
            for c in range(self._cols):
                gridspec[r,c]=self._plots[(r,c)].widget
        
        # return
        return gridspec
    
    # clear and draw with given data
    def update_tab(self,data) -> None :
        self._times_drawn+=1
        for x in self._plots.values(): x.update_plot(data)
        #print(self.name,'update_tab',self._times_drawn)

# example
SINGLE_TAB('tabname').widget

In [ ]:
'''

b=SINGLE_TAB()

b.widget

b.update_tab('asdf')

b.update_tab('alfred')

'''
None

In [ ]:
'''
doesnt contain data
only explains how to tabs within it generate plots off a pd.DataFrame
'''
class TAB_COLLECTION(IObserver,ISubject):
    def __init__(self,
        name        : str          = 'TAB_COLLECTION',
        description : str          = 'TAB_COLLECTION_DESCRIPTION',
        tabs        : TABS         = None,
        ):
        self.name=name
        self.description=description
        
        self._tabs=tabs if tabs!=None else self._populate_defaults_tabs() # contents of tab collection
        self._is_up_to_date=[True]*len(self._tabs) # nothing selected by default, all tabs up to date by default
        
        self.widget=self._make()

    ####################################
    # populate defaults
    ####################################
    def _populate_defaults_tabs(self):
        return [
            SINGLE_TAB('tab1'),
            SINGLE_TAB('tab2'),
            SINGLE_TAB('tab3'),
            SINGLE_TAB('tab4'),
        ]
    
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,info)
            
    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)
        
        if subject_name=='DATA_CONTAINER':
            self.update(subject_info['data'])

    ####################################
    # buttons
    ####################################
    '''change active tab, update as needed'''
    def _onchange_tab(self,change) -> None :
        print(self.name,':','CLICK','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')

        # handle delayed update
        if not self._is_up_to_date[self._active_tab_index]:
            print(self.name,':','UPDATE[DELAYED]','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')
            self._active_tab.update_tab(self._data) # update this tab
            self._is_up_to_date[self._active_tab_index]=True # flag tab as up to date
            
        # logging
        self._logging()
        
    ####################################
    # GUI
    ####################################
    '''build tab collection'''
    def _make(self) -> None :
        # make tab collection
        out=Tab(
            children=[x.widget for x in self._tabs],
            layout=Layout(width='100%',height='auto',min_height='500px')
        )

        # name tabs
        for i,tab_name in enumerate(x.name for x in self._tabs):
            out.set_title(index=i,title=tab_name)
            
        # observe active tab
        out.observe(self._onchange_tab,names='selected_index')

        # return
        return out

    '''delayed update of tab collection, update active tab immediate, delay update of non-active tabs till when they are viewed'''
    def delayed_full_update(self,data : pd.DataFrame) -> None:
        # only update active tab, delay update of inactive tabs till they become active
        for i,tab in enumerate(self._tabs):
            if self._active_tab_index==i:
                print(self.name,':','UPDATE[INSTANT]','NAME[',self._active_tab.name,'] INDEX[',self._active_tab_index,']')
                tab.update_tab(data) # update this particular tab
                self._is_up_to_date[i]=True # flag is up to do date
            else:
                self._is_up_to_date[i]=False # flag these tabs as out of date, update when viewed
            
        # logging
        self._logging()
    
    @property
    def _active_tab_index(self) -> int :
        return self.widget.selected_index
    
    @property
    def _active_tab(self) -> str :
        return self._tabs[self._active_tab_index]
    
    @property
    def _times_drawn(self) -> List[int]:
        return [x._times_drawn for x in self._tabs]
    
    def _logging(self) -> None:
        # delayed updating
        print(self.name,':','compute times',self._times_drawn)
        print(self.name,':','is up to date',self._is_up_to_date)

# example
TAB_COLLECTION().widget

In [ ]:
# example
a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB('a'),
        SINGLE_TAB('b'),
        SINGLE_TAB('c'),
        SINGLE_TAB('d'),
    ]
)
a.widget

In [ ]:
# global delayed update
a.delayed_full_update('abc')

In [ ]:
# request immediate update of specific tab
a._tabs[0].update_tab('asdf')

In [ ]:
a._tabs[0].update_tab('1')

In [ ]:
a._tabs[2].update_tab('2')

In [ ]:
a._tabs[2].update_tab('3')

In [ ]:
a._tabs[3].update_tab('4')

In [ ]:
a._times_drawn